In [21]:
import rail

import tables_io
import qp
import numpy as np
import h5py
import os

from rail.evaluation.dist_to_dist_evaluator import DistToDistEvaluator
from rail.evaluation.dist_to_point_evaluator import DistToPointEvaluator
from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator
from rail.evaluation.single_evaluator import SingleEvaluator
from rail.core.stage import RailStage
from rail.core.data import QPHandle, TableHandle, QPOrTableHandle

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

In [2]:
stage_dict = dict(
    metrics=['cvm', 'ks', 'rmse', 'kld', 'ad'],
    _random_state=None,
)

dtd_stage = DistToDistEvaluator.make_stage(name='dist_to_dist', **stage_dict)
#dtd_stage_single = DistToDistEvaluator.make_stage(name='dist_to_dist', force_exact=True, **stage_dict)

In [11]:
# file = h5py.File(pdfs_file, 'r')
# file.keys

<KeysViewHDF5 ['ancil', 'data', 'meta']>

In [26]:
name_ls = ['BOSS', 'DEEP2', 'GAMA', 'HSC', 'VVDSf02', 'zCOSMOS']
pivot_ls = ["1.0", "1.4"] 

ztrue_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/output_lsstErr_testSet_posts.hdf5"
ztrue_data = DS.read_file(key='data', handle_class=QPHandle, path=ztrue_file)

# TrainZ

In [ ]:
## HAS RUN ## 

In [24]:
eval_dir_TrainZ = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_TrainZ/evaluation"
os.chdir(eval_dir_TrainZ)

for i in name_ls:
    dir_0 = i
    parent_dir_0 = eval_dir_TrainZ
    path_0 = os.path.join(parent_dir_0, dir_0)
    os.makedirs(path_0, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_TrainZ/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_TrainZ/outputs/"+i+"/output_estimate_TrainZ.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

    

Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.
Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist
Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist
Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist
Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.


...

NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist
Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


In [28]:
eval_dir_invz_TrainZ = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_TrainZ/evaluation"
os.chdir(eval_dir_invz_TrainZ)

for i in pivot_ls:
    dir_0_invz = i
    parent_dir_0_invz = eval_dir_invz_TrainZ
    path_0_invz = os.path.join(parent_dir_0_invz, dir_0_invz)
    os.makedirs(path_0_invz, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_TrainZ/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_TrainZ/outputs/"+i+"/output_estimate_TrainZ.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.
Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist
Requested metrics: ['cvm', 'ks', 'rmse', 'kld', 'ad']
Processing 0 running evaluator on chunk 0 - 10000.


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


Inserting handle into data store.  output_dist_to_dist: inprogress_output_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  summary_dist_to_dist: inprogress_summary_dist_to_dist.hdf5, dist_to_dist
Inserting handle into data store.  single_distribution_summary_dist_to_dist: inprogress_single_distribution_summary_dist_to_dist.hdf5, dist_to_dist


NOTE/WARNING: Expected output file single_distribution_summary_dist_to_dist.hdf5 was not generated.


# CMNN

In [ ]:
eval_dir_CMNN = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_CMNN/evaluation"
os.chdir(eval_dir_CMNN)

for i in name_ls:
    dir_1 = i
    parent_dir_1 = eval_dir_CMNN
    path_1 = os.path.join(parent_dir_1, dir_1)
    os.makedirs(path_1, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_CMNN/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_CMNN/outputs/"+i+"/output_estimate_CMNN.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

In [ ]:
eval_dir_invz_CMNN = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_CMNN/evaluation"
os.chdir(eval_dir_invz_CMNN)

for i in pivot_ls:
    dir_1_invz = i
    parent_dir_1_invz = eval_dir_invz_CMNN
    path_1_invz = os.path.join(parent_dir_1_invz, dir_1_invz)
    os.makedirs(path_1_invz, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_CMNN/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_CMNN/outputs/"+i+"/output_estimate_CMNN.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

# GPz

In [ ]:
eval_dir_GPz = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_GPz/evaluation"
os.chdir(eval_dir_GPz)

for i in name_ls:
    dir_2 = i
    parent_dir_2 = eval_dir_GPz
    path_2 = os.path.join(parent_dir_2, dir_2)
    os.makedirs(path_2, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_GPz/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_GPz/outputs/"+i+"/output_estimate_GPz.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

In [ ]:
eval_dir_invz_GPz = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_GPz/evaluation"
os.chdir(eval_dir_invz_GPz)

for i in pivot_ls:
    dir_2_invz = i
    parent_dir_2_invz = eval_dir_invz_GPz
    path_2_invz = os.path.join(parent_dir_2_invz, dir_2_invz)
    os.makedirs(path_2_invz, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_GPz/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_GPz/outputs/"+i+"/output_estimate_GPz.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

# PZFlow

In [ ]:
eval_dir_PZFlow = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_PZFlow/evaluation"
os.chdir(eval_dir_PZFlow)

for i in name_ls:
    dir_3 = i
    parent_dir_3 = eval_dir_PZFlow
    path_3 = os.path.join(parent_dir_3, dir_3)
    os.makedirs(path_3, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_PZFlow/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_PZFlow/outputs/"+i+"/output_estimate_PZFlow.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

In [ ]:
eval_dir_invz_PZFlow = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_PZFlow/evaluation"
os.chdir(eval_dir_invz_PZFlow)

for i in pivot_ls:
    dir_3_invz = i
    parent_dir_3_invz = eval_dir_invz_PZFlow
    path_3_invz = os.path.join(parent_dir_3_invz, dir_3_invz)
    os.makedirs(path_3_invz, exist_ok=True)

    os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_PZFlow/evaluation/"+i)
    pdfs_file = "/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_PZFlow/outputs/"+i+"/output_estimate_PZFlow.hdf5"
    ensemble = DS.read_file(key='data', handle_class=QPHandle, path=pdfs_file)

    dtd_results = dtd_stage.evaluate(ztrue_data, ensemble)

# FZBoost

In [ ]:
os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/specSelection_lsstErr_FZBoost/evaluation")

In [ ]:
os.chdir("/Users/alicec03/Desktop/Summer_Research/Photo-z-Stress-Test/Photo-z-Stress-Test/invz_lsstErr_FZBoost/evaluation")